In [1]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
import warnings
import seaborn as sns
from sklearn.linear_model import LinearRegression

### Time related preprocessing
- fetch from BigQuery
    - DATETIME_TRUNC: cut data
    - EXTRACT => MONTH, WEEK, DAY, HOUR can be extracted
    - FORMAT_DATETIME => which day can be extracted (1 = Monday, 0 = Monday in Pandas)

In [2]:
import os
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/workspace/Data-Science/mobility-285808-aa8e44818175.json'

# import GCP Client library
from google.cloud import bigquery

#instantiate a client for bigquery service
bigquery_client = bigquery.Client()
bigquery_client

In [3]:
query = """
SELECT
    *,
    EXTRACT(MONTH FROM pickup_hour) AS month,
    EXTRACT(DAY FROM pickup_hour) AS day,
    CAST(format_datetime('%u', pickup_hour) AS INT64) -1 AS weekday,
    EXTRACT(HOUR FROM pickup_hour) AS hour,
    CASE WHEN CAST(FORMAT_DATETIME('%u', pickup_hour) AS INT64) IN (6, 7) THEN 1 ELSE 0 END AS is_weekend
FROM (
    SELECT 
        DATETIME_TRUNC(pickup_datetime, hour) AS pickup_hour,
        count(*) AS cnt
    FROM `bigquery-public-data.new_york_taxi_trips.tlc_yellow_trips_2015` 
    WHERE EXTRACT(MONTH from pickup_datetime) = 1
    GROUP BY pickup_hour
)
ORDER BY pickup_hour
"""

In [4]:
# run query and get data from bigquery
query_job = bigquery_client.query(query) #API request

In [5]:
# data into dataframe
df = query_job.to_dataframe()
df.head()

,pickup_hour,cnt,month,day,weekday,hour,is_weekend
0,2015-01-01 00:00:00,28312,1,1,3,0,0
1,2015-01-01 01:00:00,31707,1,1,3,1,0
2,2015-01-01 02:00:00,28068,1,1,3,2,0
3,2015-01-01 03:00:00,24288,1,1,3,3,0
4,2015-01-01 04:00:00,17081,1,1,3,4,0


In [6]:
df.tail()

,pickup_hour,cnt,month,day,weekday,hour,is_weekend
739,2015-01-31 19:00:00,32436,1,31,5,19,1
740,2015-01-31 20:00:00,27555,1,31,5,20,1
741,2015-01-31 21:00:00,27477,1,31,5,21,1
742,2015-01-31 22:00:00,29862,1,31,5,22,1
743,2015-01-31 23:00:00,29856,1,31,5,23,1


In [7]:
query = """
SELECT
    DATETIME_TRUNC(pickup_datetime, hour) AS pickup_hour,
    EXTRACT(MONTH FROM pickup_datetime) AS month,
    EXTRACT(DAY FROM pickup_datetime) AS day,
    CAST(FORMAT_DATETIME('%u', pickup_datetime) AS INT64) -1 AS weekday,
    EXTRACT(HOUR FROM pickup_datetime) AS hour,
    CASE WHEN CAST(FORMAT_DATETIME('%u', pickup_datetime) AS INT64) IN (6, 7) THEN 1 ELSE 0 END AS is_weekend,
    count(*) AS cnt
FROM `bigquery-public-data.new_york_taxi_trips.tlc_yellow_trips_2015`
WHERE EXTRACT(MONTH from pickup_datetime) = 1
GROUP BY pickup_hour, month, day, weekday, hour, is_weekend
ORDER BY pickup_hour
"""

In [8]:
# run query and get data from bigquery
query_job = bigquery_client.query(query) #API request

# data into dataframe
df = query_job.to_dataframe()
df.head()

,pickup_hour,month,day,weekday,hour,is_weekend,cnt
0,2015-01-01 00:00:00,1,1,3,0,0,28312
1,2015-01-01 01:00:00,1,1,3,1,0,31707
2,2015-01-01 02:00:00,1,1,3,2,0,28068
3,2015-01-01 03:00:00,1,1,3,3,0,24288
4,2015-01-01 04:00:00,1,1,3,4,0,17081


In [9]:
# extract using python, pd.to_datetime, ex. df[col].df.xxx
exact_query = """
SELECT
    DATETIME_TRUNC(pickup_datetime, hour) AS pickup_hour,
    count(*) AS cnt
FROM `bigquery-public-data.new_york_taxi_trips.tlc_yellow_trips_2015`
WHERE EXTRACT(MONTH from pickup_datetime) = 1
GROUP BY pickup_hour
ORDER BY pickup_hour
"""

In [10]:
# run query and get data from bigquery
query_job = bigquery_client.query(exact_query) #API request

# data into dataframe
df = query_job.to_dataframe()
df.head()

,pickup_hour,cnt
0,2015-01-01 00:00:00,28312
1,2015-01-01 01:00:00,31707
2,2015-01-01 02:00:00,28068
3,2015-01-01 03:00:00,24288
4,2015-01-01 04:00:00,17081


In [11]:
 df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 744 entries, 0 to 743
Data columns (total 2 columns):
pickup_hour    744 non-null datetime64[ns]
cnt            744 non-null int64
dtypes: datetime64[ns](1), int64(1)
memory usage: 11.8 KB


In [16]:
df['pickup_hour'].dt.date

0      2015-01-01
1      2015-01-01
2      2015-01-01
3      2015-01-01
4      2015-01-01
          ...    
739    2015-01-31
740    2015-01-31
741    2015-01-31
742    2015-01-31
743    2015-01-31
Name: pickup_hour, Length: 744, dtype: object

In [17]:
df['pickup_hour'].dt.hour

0       0
1       1
2       2
3       3
4       4
       ..
739    19
740    20
741    21
742    22
743    23
Name: pickup_hour, Length: 744, dtype: int64

In [18]:
df['month'] = df['pickup_hour'].dt.month
df['day'] = df['pickup_hour'].dt.day
df['weekday'] = df['pickup_hour'].dt.weekday
df['hour'] = df['pickup_hour'].dt.hour
df['is_weekend'] = (df['pickup_hour'].dt.weekday // 5 == 1).astype(int)

In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 744 entries, 0 to 743
Data columns (total 7 columns):
pickup_hour    744 non-null datetime64[ns]
cnt            744 non-null int64
month          744 non-null int64
day            744 non-null int64
weekday        744 non-null int64
hour           744 non-null int64
is_weekend     744 non-null int64
dtypes: datetime64[ns](1), int64(6)
memory usage: 40.8 KB


In [20]:
df.tail()

,pickup_hour,cnt,month,day,weekday,hour,is_weekend
739,2015-01-31 19:00:00,32436,1,31,5,19,1
740,2015-01-31 20:00:00,27555,1,31,5,20,1
741,2015-01-31 21:00:00,27477,1,31,5,21,1
742,2015-01-31 22:00:00,29862,1,31,5,22,1
743,2015-01-31 23:00:00,29856,1,31,5,23,1


### BigQuery GIS, Reverse Geocoding

- coordinates => transfer to 'zip_code'
- BigQuery, bigquery-public-data.geo_us_boundaries.zip_codes
- use ST_CONTAINS to extract data